In [62]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
import language_tool_python
##  StratifiedKFold、cross_val_scoreをインポート
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score


In [63]:
test = pd.read_csv('../input/test_essays.csv')
sub = pd.read_csv('../input/sample_submission.csv')
org_train = pd.read_csv('../input/train_essays.csv')

train = pd.read_csv("../input/train_v2_drcat_02.csv", sep=',')

train = train.drop_duplicates(subset=["text"])

train.reset_index(drop=True)
x_train = train["text"].values
y_train = train["label"].values

print(x_train)



["Phones\n\nModern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving. They are some really bad consequences when stuff happens when it comes to a phone. Some certain areas in the United States ban phones from class rooms just because of it.\n\nWhen people have phones, they know about certain apps that they have .Apps like Facebook Twitter Instagram and Snapchat. So like if a friend moves away and you want to be in contact you can still be in contact by posting videos or text messages. People always have different ways how to communicate with a phone. Phones have changed due to our generation.\n\nDriving is one of the way how to get around. People always be on their phones while doing it. Which can cause serious Problems. That's why there's a thing that's called no texting while driving. That's a really important thing to rem

In [64]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs = x_train

bag = count.fit_transform(docs)
print(bag.toarray().shape)


(44868, 77326)


In [65]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer(text):
    return text.split()


def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]


In [66]:
import nltk

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/matsuisouta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [67]:
from nltk.corpus import stopwords

stop = stopwords.words('english')


In [68]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDFベクトル化器を作成
tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

# ロジスティック回帰モデルの作成
lr = LogisticRegression(penalty='l2', C=1.0, random_state=0)

# パイプラインを作成
lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', lr)])

# データを使ってモデルを訓練
lr_tfidf.fit(x_train, y_train)


Pipeline(steps=[('vect', TfidfVectorizer(lowercase=False)),
                ('clf', LogisticRegression(random_state=0))])

In [69]:
# 層化K分割交差検証を設定
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# データを使って交差検証を実行
scores = cross_val_score(lr_tfidf, x_train, y_train, cv=skf, scoring='roc_auc')

# 各フォールドでのAUCを表示
for fold, score in enumerate(scores, start=1):
    print(f'Fold {fold}: AUC = {score:.4f}')


Fold 1: AUC = 0.9993
Fold 2: AUC = 0.9989
Fold 3: AUC = 0.9991
Fold 4: AUC = 0.9992
Fold 5: AUC = 0.9992


In [70]:
# AUCの平均値を表示
print('Mean AUC = {:.4f}'.format(scores.mean()))


Mean AUC = 0.9991


In [71]:
# テストデータを予測
y_test = lr_tfidf.predict_proba(test['text'])
print(y_test)


[[0.09592568 0.90407432]
 [0.09592568 0.90407432]
 [0.09592568 0.90407432]]


In [72]:
# org_trainのデータを予測
y_org_train = lr_tfidf.predict_proba(org_train['text'])
print(y_org_train[:30, 1])


[0.00128612 0.04733991 0.04967885 0.01967201 0.01280526 0.00126446
 0.0041796  0.00612767 0.00414179 0.01347454 0.00767354 0.1254411
 0.00413933 0.00702183 0.00506423 0.08154811 0.00444207 0.0391298
 0.00145596 0.00466394 0.0183595  0.20946305 0.00271278 0.03190676
 0.00536021 0.0790171  0.01450786 0.00413084 0.03099834 0.00764798]


In [73]:
#kaggleの提出用ファイルを作成
sub['generated'] = y_test[:, 1]
sub.to_csv('submission.csv', index=False)
